<H1> FedPROX


Connect the Notebook with GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/FL2022/FedPROX')

Import Libraries

In [ ]:
from options import args_parser
from update import LocalUpdate, test_inference, client_update, server_aggregate, test
from utils import get_dataset, average_weights, weighted_average_weights, exp_details, get_n_params
from modelRN50 import ResNet50
import torch
torch.cuda.empty_cache()
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm # progress bar
import copy
from torch.utils.tensorboard import SummaryWriter
logger = SummaryWriter('../logs')

Set the parameters for the training 

In [ ]:
sys.argv=['',
          '--iid=0',  #0 -> NONiid, 1 -> iid
          '--num_users=100',
          '--lr=0.0001',
          '--local_ep=5',
          '--epochs=10',
          '--optimizer=adam',
          '--norm=batch_norm',
          '--local_bs=10',
          '--dataset=cifar',
          '--loss=CrossEntropyLoss',
          '--gpu=/device:GPU:0']
args=args_parser()
num_selected = int( args.num_users * args.frac )
baseline_num = 100 # number of baseline images to be saved on the global server
                   # for retraining of the client's model before aggregation
unbalanced = False # if True the clients will contain different number of classes
verbose = False
mu=0.01


Set the device on cuda

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if verbose:
 print(f"Device available: {device}")

###Build the Model ResNet-50

In [ ]:
global_model = ResNet50(args.norm)
global_model.to(device)
global_model.train()
if verbose:
  print(global_model)
  print(f"Number of Parameters: {get_n_params(global_model)}")

global_weights = global_model.state_dict()
client_models = [ global_model for _ in range(num_selected)]

# Synchronizing the clients with the global model 
for model in client_models:
    model.load_state_dict(global_model.state_dict()) 
# Optimizer selection
if args.optimizer == 'sgd':
    opt = [optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=args.wd) for model in client_models]
elif args.optimizer == 'adam':
    opt = [optim.Adam(model.parameters(), lr=args.lr,  weight_decay=args.wd) for model in client_models]

####Dataset split:

In [ ]:
train_dataset, test_dataset, user_groups = get_dataset(args=args, unbalanced=unbalanced,)

#Training

In [ ]:
# training
train_loss, train_accuracy = [], []
test_acc_list, test_loss_list = [], []

for epoch in range(1, args.epochs+1):
    local_weights = []
    local_losses = []
    print(f'Epoch: {epoch} \n')
    model.train()
    m = max(int(args.frac * args.num_users), 1) # num of users, at least 1
    idxs_users = np.random.choice(range(args.num_users), m, replace=False) 

    for idx in idxs_users: 
        local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[idx],logger=logger, mu=mu) # consider in the local update the parameter mu
        w, loss = local_model.update_weights(model=copy.deepcopy(model), # pass the global model to the clients
                                             global_round=epoch)
        print('| Client : {} | Average Loss: {:.4f} '.format(idx, loss))
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # compute global weights (average of local weights)
    if unbalanced:
        global_weights = weighted_average_weights(local_weights, user_groups, idxs_users)
    else:
        global_weights = average_weights(local_weights)

    # update weights of the global model
    model.load_state_dict(global_weights)

    # compute average loss
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    model.eval()

    # calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    for client in range(args.num_users):
        local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[client],logger=logger, mu=mu)
        acc, loss = local_model.inference(model=model)
        list_acc.append(acc)
        list_loss.append(loss)

    
    train_accuracy.append(sum(list_acc)/len(list_acc))
    print(f'\nAverage training statistics (global epoch) : {epoch}')
    print(f'|---- Trainig Loss : {np.mean(np.array(train_loss))}')
    print('|---- Training Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))
    test_loss, test_acc = test(args, model,global_model, test_dataset)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    print('%d-th round' %epoch)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


Storing the Results

In [ ]:
dict_ = {'train_acc' : train_accuracy, 'train_loss' : train_loss}
df = pd.DataFrame(dict_) 
iid = ['iid' if args.iid else 'nonIID']
unb = ['unbalanced' if unbalanced and not args.iid else 'balanced' ]
bs = args.local_bs 
filename = f"fedPROX_{iid}_{unb}_{args.norm}{bs}_{args.epochs}_lr_{args.lr}_optimizer_{args.optimizer}"
df = pd.DataFrame(dict_) 
df.to_csv('/content/drive/MyDrive/FL2022/FedPROX/Results/'+filename+'.csv', encoding='utf-8')

Showing the Results

In [ ]:
print(f' \n Results after {args.epochs} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
